# Imports

In [1]:
import scipy.io as sio
import os
from DenoisingGCN.GCNModel import DGCNN
import numpy as np
import torch
import meshplot as mp

# Matlab files content
*What is inside the Matlab files that are imported by the GCN?* The structure of the files are printed and defined as follows:

File name: [INT]_[INT].mat<br>
item: MAT, ([N], [N])<br>
item: FEA, (17, [N])<br>
item: GT, (3, 1)<br>
item: NOR, (3, 1)<br>

where,<br>
[INT] needs to be replaced by an integer.<br>
[N] the amount of nodes in the graph of the patch.<br>
MAT represents an adjacency matrix telling which node of the graph is connected to which other node by the inputs 0 and 1.<br>
FEA represents the feature vectors (one per node). There are 17 features in total! (3 for the face center, 3 for the face normal, 1 for the face area, 1 for the number of neighbours and 9 for the vertex positions of the center face).<br>
GT represents the expected ground truth normal vector of the center face of the patch.<br>
NOR represents the current center normal.

In [2]:
for filename in os.listdir('.'):
    filepath = './' + filename
    if not filepath.endswith(".mat"):
        continue
    annots = sio.loadmat(filepath)
    
    itemprint = f"File name: {filename}\n"
    for i, v in annots.items():
        if i[0] != "_":
            itemprint += f"item: {i}, {v.shape}\n"
        if i == "FEA":
            n = 1
            itemprint += f"value: {v[(3*n):(3*(n+1)), 0]}\n"
        if i == "GT":
            itemprint += f"value: {v.reshape(-1)}\n"
        if i == "NOR":
            itemprint += f"value: {v.reshape(-1)}\n"
    print(itemprint)

File name: example_file.mat
item: MAT, (69, 3)
item: FEA, (69, 17)
value: [0.11576948 0.089921   0.11675524]
item: GT, (69, 3)
value: [ 0.99725868  0.00302159  0.07393236  0.98767921  0.02141124 -0.15502047
  0.98423639 -0.17676412  0.00575992  0.93651318 -0.1169218  -0.33056368
  0.92257508 -0.33338995 -0.19418124  0.9289599  -0.33687683 -0.15345197
  0.84948937 -0.52499991 -0.05237274  0.92219692  0.29156043 -0.25405779
  0.92616434  0.29357551 -0.23671298  0.98046906 -0.09328901 -0.17314035
  0.99264052 -0.11079499 -0.04888009  0.93014105 -0.36701155 -0.01183847
  0.92853915 -0.37114112  0.00832567  0.96701949 -0.24755102 -0.05993154
  0.95945296 -0.27110525  0.0771489   0.85871305 -0.51053942 -0.04428771
  0.86408506 -0.48443215 -0.13668396  0.9236361  -0.33133807 -0.19264327
  0.92679711 -0.37523567 -0.01566251  0.95348865 -0.293639   -0.0680847
  0.94576251 -0.31853791  0.0637721   0.99123536 -0.13089077 -0.01789034
  0.98966403 -0.12691151 -0.06677263  0.98084889  0.16310635 -0.

# Passing a matlab file through the GCN.
*How do we pass the input data of a Matlab file throughn the GCN?* First we need to instantiate a GCN model in the GPU (or maybe CPU???) and load the weights with a model:

In [3]:
dgcnn = DGCNN(8, 17, 1024, 0.5)
dgcnn.load_state_dict(torch.load("DenoisingGCN/checkpoints/4_model.t7"))
dgcnn.cuda()
dgcnn.eval()
print("Loading DGCNN in GPU complete!")

Loading DGCNN in GPU complete!


After that, we need to load the Matlab file. (Every file represents a patch, so we are loading in the patch graph and we're trying to pass it through the GCN) For this, we first load the loadMAT definition copied from datautils.MatrixDataset. This loads a file and adds padding or removes values so that all patches have the same amount of faces represented in the patch.

In [4]:
# This method is copied from datautils.MatrixDataset, because the class / definition cannot be imported from this location

def loadMAT(data_path, num_neighbors):
    source_data = sio.loadmat(data_path)
    input_matrix = source_data["MAT"]
    input_matrix = np.array(input_matrix)
    
    input_features = source_data["FEA"]
    input_features = np.array(input_features)
    input_features = input_features.T

    num_faces = input_matrix.shape[0]
    if(num_faces >= num_neighbors):
        input_matrix = input_matrix[0:num_neighbors, 0:num_neighbors]
        input_features = input_features[0:num_neighbors]
    else:
        # Matrix Padding
        input_matrix = np.pad(input_matrix, ((0, num_neighbors - num_faces), (0, num_neighbors - num_faces)), \
            'constant', constant_values = (0, 0))
        input_features = np.pad(input_features, ((0, num_neighbors - num_faces), (0, 0)), \
            'constant', constant_values = (0, 0))

    input_indices = []
    for i in range(num_neighbors):
        temp_idx = np.array((input_matrix[i] == 1).nonzero()).reshape(-1)
        temp_idx = list(temp_idx)
        if(len(temp_idx) == 0):
            temp_idx = [num_neighbors - 1, num_neighbors - 1, num_neighbors - 1]
        elif(len(temp_idx) == 1):
            temp_idx.append(temp_idx[0])
            temp_idx.append(temp_idx[0])
        elif(len(temp_idx) == 2):
            temp_idx.append(temp_idx[1])

        input_indices.append(temp_idx)

    input_indices = np.array(input_indices)

    gt_norm = source_data["GT"]
    gt_norm = np.array(gt_norm).reshape(-1).astype(np.float32)

    center_norm = source_data["NOR"]
    center_norm = np.array(center_norm).reshape(-1).astype(np.float32)
    
    gt_res = ((np.dot(gt_norm, center_norm) * gt_norm) - center_norm + 1.) / 2.

    inputs = np.c_[input_features, input_indices]

    # print(f"inputs: {inputs.shape}\nfeatures: {input_features.shape}\nindices: {input_indices.shape}")

    return inputs, gt_res, gt_norm, center_norm

Finally, we're gonna load an example file, pass it through the GCN and print the guessed normal vector! As can be seen in the result, the network doesn't understand that normal vectors need to have a unit length. However we can later fix this ourselves.

In [5]:
data = loadMAT("./samples/example_object/0_0.mat", 64)
inputs, gt_res, gt_norm, center_norm = data
inputs = torch.from_numpy(inputs[None, :, :])
inputs = inputs.type(torch.FloatTensor)
inputs = inputs.permute(0, 2, 1)
inputs = inputs.cuda()
outputs = dgcnn(inputs)
normal = outputs.cpu().detach().numpy().reshape(-1)
print(f"Predicted normal vector: {normal}\nLength: {np.linalg.norm(normal)}")

Predicted normal vector: [ 0.96460986 -0.01689533  0.00228474]
Length: 0.9647605419158936


One patch can be passed through the GCN. However, the implementation is heavily based on batches, so to not make use of batches is a bad practice. This works as a proof of concept though!

# Visualize patches from files
Apparently all data needed to visualize the patch that a Matlab file represents is in the files. To check if this is true, we can try to visualize individual files and explore what the given example files look like. The visualization below shows what patches look like with a point cloud. Showing it as a mesh would be better, but the data structure of the feature files are not good enough to make a robust visualization.

In [6]:
source_data = sio.loadmat("./samples/Test_Bunny/0_100.mat")
mat = source_data['MAT']
fea = source_data['FEA'].T
points = fea[:, 8:11]
centers = fea[:, 0:3]

centers_and_vertices = np.concatenate((centers, points), axis=0)
color = np.ones(fea.shape[0] * 2)
color[0:fea.shape[0]] = 0
mp.plot(centers_and_vertices, c=color, shading={"point_size": 0.1})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4313424…

A proof of the last 9 features in a feature node being the same 3 points can be seen below:

In [7]:
points = fea[:, 8:17]
first_points = points[:, 0:3]
second_points = points[:, 3:6]
third_points = points[:, 6:9]
print(f"First and second points are the the same: {(first_points == second_points).all()}")
print(f"First and third points are the the same: {(first_points == third_points).all()}")
print(f"Second and third points are the the same: {(second_points == third_points).all()}")

First and second points are the the same: True
First and third points are the the same: True
Second and third points are the the same: True


# Create and save Matlab patch files
I think it's time to create new Matlab patch files! This will enable us to make networks learn on more models instead of only the example patch files. First, we need to import the functionality that can create patches for us.

In [8]:
from PatchGeneration.Modules.PatchCollector import PatchCollector as pc
from PatchGeneration.Modules.Mesh import Mesh

Then, we can open a file and create an example patch:

In [9]:
EXAMPLE_FILE_NAME = "models/armadillo.obj"
EXAMPLE_FACE_INDEX = 0

example_mesh = Mesh.readFile(EXAMPLE_FILE_NAME)
example_collector = pc(example_mesh)
example_patch = example_collector.selectPaperPatch(EXAMPLE_FACE_INDEX)
example_patch.alignPatch()
mp.plot(example_patch.v, example_patch.f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0244394…

From this example patch, we want to create the required "MAT" and "GT" 2D arrays. After this, noise can be applied and the "FEA" 2D feature vectors can be created. (GT should be the Ground Truth normal vectors of the faces) (We import IGL for calculating the adjacency matrix. The matrix exists for the full mesh, but not for single patches)

In [10]:
import igl

In [11]:
MAT = igl.triangle_triangle_adjacency(example_patch.f)[0]
GT = example_patch.getFaceNormals()
NOR = np.zeros((MAT.shape[0], 3)) # This is a placeholder, since we're not gonna use it.

# Apply noise to model

# Create feature vector

centers = igl.barycenter(example_patch.v, example_patch.f)
normals = example_patch.getFaceNormals()
areas = example_patch.getAreas(np.arange(example_patch.f.shape[0])).reshape(-1, 1)
neighbors = (MAT != -1).sum(axis=1).reshape(-1, 1)
point_positions = example_patch.v[example_patch.f].reshape(-1, 9)

FEA = np.concatenate((centers, normals, areas, neighbors, point_positions), axis=1)
print(FEA.shape)
file_dictionary = {
    "MAT": MAT,
    "FEA": FEA,
    "GT": GT,
    "NOR": NOR
}

sio.savemat("example_file.mat", file_dictionary)

(69, 17)


# Create Matlab files for Mesh
Not only do we want to create files for a single patch, but ideally, we want to create a pipeline that creates a file for every patch of the mesh. This functionality also needs to be made!

In [12]:
import os
import time

In [13]:
my_collector = pc.readFile("PatchGeneration/Object/example_object.obj")
my_collector.collectAndSavePatches("samples/example_object", timeout=1)

Start selecting patches
[Timeout: 0/1] Patch 1/44694 selected!
[Timeout: 0/1] Patch 2/44694 selected!
[Timeout: 0/1] Patch 3/44694 selected!
[Timeout: 0/1] Patch 4/44694 selected!
[Timeout: 0/1] Patch 5/44694 selected!
[Timeout: 0/1] Patch 6/44694 selected!
[Timeout: 0/1] Patch 7/44694 selected!
[Timeout: 0/1] Patch 8/44694 selected!
[Timeout: 0/1] Patch 9/44694 selected!
[Timeout: 0/1] Patch 10/44694 selected!
[Timeout: 0/1] Patch 11/44694 selected!
[Timeout: 0/1] Patch 12/44694 selected!
[Timeout: 0/1] Patch 13/44694 selected!
[Timeout: 0/1] Patch 14/44694 selected!
[Timeout: 0/1] Patch 15/44694 selected!
[Timeout: 0/1] Patch 16/44694 selected!
[Timeout: 0/1] Patch 17/44694 selected!
[Timeout: 0/1] Patch 18/44694 selected!
[Timeout: 0/1] Patch 19/44694 selected!
[Timeout: 0/1] Patch 20/44694 selected!
[Timeout: 0/1] Patch 21/44694 selected!
[Timeout: 0/1] Patch 22/44694 selected!
[Timeout: 0/1] Patch 23/44694 selected!
[Timeout: 0/1] Patch 24/44694 selected!
[Timeout: 0/1] Patch 25/4

### Calculating noise
In the codeblock below, a method for calculating noise for *N* vertices has been created. If all vertices are updated, new face normals need to be calculated!!!

In [14]:
N = 10
random_sample = np.random.random((N, 3))
random_direction = random_sample / np.linalg.norm(random_sample, axis=1)[:, None]
random_gaussian_sample = np.random.normal(0, 1, (N, 1))
noise = random_direction * np.tile(random_gaussian_sample, (1, 3))
print(random_direction.shape, random_gaussian_sample.shape, noise.shape)
print(np.concatenate((noise, random_gaussian_sample), axis=1))

(10, 3) (10, 1) (10, 3)
[[ 0.05445699  0.0458581   0.02660534  0.07600245]
 [ 0.69575272  0.56869001  0.48226587  1.01983358]
 [-0.81924196 -0.58090892 -0.48177507 -1.11387602]
 [-0.39210559 -0.36343132 -0.11611422 -0.5470938 ]
 [ 1.13885941  0.13930076  0.92827327  1.47583763]
 [-0.89999237 -0.37524597 -0.02605682 -0.97543568]
 [-0.45211319 -0.15089591 -0.00742847 -0.47668763]
 [ 0.06841851  0.30017828  0.76078972  0.82072473]
 [-0.02801378 -0.09321038 -0.06365431 -0.11629626]
 [-0.27777374 -0.07708141 -0.35253838 -0.45539335]]


### Displaying Gaussian Noise example
Below is a piece of code, which is used to visualize the difference between the ground truth model and noisy model (To also proof that it's working)

In [16]:
LEVEL = 3
noise_factor = LEVEL / 10
a = igl.adjacency_list(example_mesh.f)
avg_dists = np.zeros(example_mesh.getVertices().shape[0])
for i, v in enumerate(a):
    pos = example_mesh.getVertices()[v] - example_mesh.getVertices()[i]
    avg_dist = np.average(np.linalg.norm(pos, axis=1))
    avg_dists[i] = avg_dist
example_mesh.applyGaussianNoise(noise_factor)
v = np.concatenate((example_mesh.v, example_mesh.noisy_v), axis=0)
v[example_mesh.v.shape[0]:, 0] += example_mesh.getPCBoundingBox()[0]
f = np.tile(example_mesh.f, (2, 1))
f[example_mesh.f.shape[0]:] += example_mesh.v.shape[0]
c = np.zeros((v.shape[0], 1))
c[example_mesh.v.shape[0]:] = 1
mp.plot(v, f, c)


Random average displacement: 0.0002724843411887766


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(63.643390…